<a href="https://colab.research.google.com/github/blue-create/langlens/blob/main/models/sentencebert%26bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [1]:
%%capture
!pip install transformers==4.20.0

In [2]:
%%capture
!pip install datasets

In [3]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import json
import spacy


#preparing
import datasets
from datasets import Dataset
from ast import literal_eval

#modelling

from transformers import TrainingArguments, Trainer
from transformers import BloomConfig, BloomModel
from transformers import AutoTokenizer

#evaluation
import torch
from transformers import EvalPrediction
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction

### Constants

In [4]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
# change cwd
%cd drive/MyDrive/Work/Frontline/data

/content/drive/.shortcut-targets-by-id/1WfnZsqpG1r110J63sMbfS5TpsDOkveiV/data


In [6]:
from scripts import annotations

### Data

In [7]:
data=pd.read_csv("annotated/230621_all_annotationsV1.csv", index_col=0,converters={"annotations":literal_eval})
data=data.reset_index(drop=True)

In [8]:
data.loc[:,"annotations"]=data.apply(annotations.ground_truth_filter,min_coannotation=1,min_similarity=-1, similarity="dice",axis=1)


In [9]:
data.annotations=data.annotations.apply(list)
data.annotations=["*".join(i) for i in data.annotations]

In [10]:
# converting the annotation column with one hot encoding
df=pd.merge(data[["text","artikel_id"]],data.annotations.str.get_dummies(sep="*"), left_index=True, right_index=True)

In [11]:
# create a list of all labels
labels=df.columns.to_list()[-4:]

### Split data into train and test

In [12]:
#split train, dev , test sets
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

#### Convert DataFrame to  Dataset

In [13]:
ds=datasets.DatasetDict({"train":Dataset.from_dict(df_train),"val":Dataset.from_dict(df_val),"test":Dataset.from_dict(df_test)})

### Tokenizing & Encoding

In [14]:
%%capture
pip install sentence_transformers

In [15]:
from sentence_transformers import SentenceTransformer

In [16]:
#tokenizer = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')
tokenizer = AutoTokenizer.from_pretrained('T-Systems-onsite/cross-en-de-roberta-sentence-transformer',is_split_into_words=True)

Downloading:   0%|          | 0.00/188 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/541 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [17]:
def preprocess_data(df):
  text = df["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels

  labels_batch = {k: df[k] for k in df.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]
  encoding["labels"] = labels_matrix.tolist()
  return encoding

In [18]:
ds_encoded=ds.map(preprocess_data, batched=True, remove_columns=ds['train'].column_names)

Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

In [19]:
ds_encoded.set_format("torch")


In [20]:
ds_encoded["train"]["input_ids"].shape

torch.Size([1024, 128])

### Modelling

In [21]:
labels

['Domestic Violence',
 'Graphic',
 'Sensationalist',
 'Statement of responsibility']

In [22]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [23]:
from transformers import AutoModelForSequenceClassification


In [24]:
model = AutoModelForSequenceClassification.from_pretrained('T-Systems-onsite/cross-en-de-roberta-sentence-transformer',
                                   problem_type="multi_label_classification",
                                   num_labels=len(labels),
                                   id2label=id2label,
                                   label2id=label2id)

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at T-Systems-onsite/cross-en-de-roberta-sentence-transformer were not used when initializing XLMRobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at T-Systems-onsite/cross-en-de-roberta-sentence-transformer and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out

### Evaluate

In [25]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):

    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [26]:
ds_encoded["train"]=ds_encoded["train"].rename_column("labels","label")
ds_encoded["test"]=ds_encoded["test"].rename_column("labels","label")
ds_encoded["val"]=ds_encoded["val"].rename_column("labels","label")


In [27]:
args = TrainingArguments(
    f"roberta-finetuned-classification",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=10,
    weight_decay=0.05,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    #push_to_hub=True,
)

In [28]:
ds_encoded

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'label'],
        num_rows: 1024
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'label'],
        num_rows: 128
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'label'],
        num_rows: 128
    })
})

In [29]:
trainer = Trainer(
    model,
    args,
    train_dataset=ds_encoded["train"],
    eval_dataset=ds_encoded["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

In [30]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1024
  Num Epochs = 10
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 1710


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.189582,0.899225,0.930447,0.906250
2,No log,0.160758,0.894118,0.924064,0.890625
3,0.187300,0.162741,0.901186,0.926681,0.890625
4,0.187300,0.167727,0.916996,0.936992,0.906250
5,0.187300,0.189146,0.906250,0.933065,0.906250
6,0.082200,0.160799,0.872000,0.904833,0.851562
7,0.082200,0.196366,0.893281,0.921526,0.882812
8,0.082200,0.212558,0.889764,0.920217,0.882812
9,0.043400,0.213428,0.859375,0.902135,0.859375
10,0.043400,0.212893,0.890625,0.922755,0.890625


***** Running Evaluation *****
  Num examples = 128
  Batch size = 6
Saving model checkpoint to roberta-finetuned-classification/checkpoint-171
Configuration saved in roberta-finetuned-classification/checkpoint-171/config.json
Model weights saved in roberta-finetuned-classification/checkpoint-171/pytorch_model.bin
tokenizer config file saved in roberta-finetuned-classification/checkpoint-171/tokenizer_config.json
Special tokens file saved in roberta-finetuned-classification/checkpoint-171/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 128
  Batch size = 6
Saving model checkpoint to roberta-finetuned-classification/checkpoint-342
Configuration saved in roberta-finetuned-classification/checkpoint-342/config.json
Model weights saved in roberta-finetuned-classification/checkpoint-342/pytorch_model.bin
tokenizer config file saved in roberta-finetuned-classification/checkpoint-342/tokenizer_config.json
Special tokens file saved in roberta-finetuned-classification/che

TrainOutput(global_step=1710, training_loss=0.09463468685484769, metrics={'train_runtime': 817.8523, 'train_samples_per_second': 12.521, 'train_steps_per_second': 2.091, 'total_flos': 673576397045760.0, 'train_loss': 0.09463468685484769, 'epoch': 10.0})

## Predicting

In [41]:
sigmoid = torch.nn.Sigmoid()

In [42]:
test_pred=pd.DataFrame(ds["test"])

In [43]:
def predict_text(text):
  encoding = tokenizer(text, return_tensors="pt")
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
  outputs = trainer.model(**encoding)
  logits = outputs.logits
  probs = sigmoid(logits.squeeze().cpu())
  predictions = np.zeros(probs.shape)
  predictions[np.where(probs >= 0.5)] = 1
  # turn predicted id's into actual label names
  predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
  return predicted_labels


In [44]:
pred=[]
for t in ds["test"]["text"]:
  pred.append(predict_text(t))
test_pred["prediction"]=pred

### Calculate Performance
Calculate Hamming score as indication for predictions performance.

In [45]:
temp=pd.DataFrame(columns=["Domestic Violence",	"Graphic",	"Sensationalist",	"Statement of responsibility"])

In [46]:
def hamming_score(y_true, y_pred):
    return (
        (y_true & y_pred).sum(axis=1) / (y_true | y_pred).sum(axis=1)
    ).mean()


In [47]:
predicted_labels=pd.DataFrame(pred)[0].str.get_dummies()
predicted_labels=pd.concat([temp,predicted_labels]).fillna(0)
actual_labels=test_pred.iloc[:,-5:-1]

In [48]:
hamming_score(actual_labels, predicted_labels)

0.8828125

Performance for Articles not labelled "Domestic Violence"

In [49]:
predicted_labels=predicted_labels[test_pred["Domestic Violence"]!=1]
actual_labels=actual_labels[test_pred["Domestic Violence"]!=1]

In [50]:
hamming_score(actual_labels, predicted_labels)

0.09090909090909091